In [ ]:
# First version, may contain significant errors and examples of extremal incompetence of the autor

In [664]:
import sys

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Perceptron
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np
import pandas as pd

In [665]:
languages_data_folder = 'data/languages/paragraphs/'
dataset = load_files(languages_data_folder)

In [666]:
dft = pd.DataFrame(data = dataset.target, columns = ['data'])

In [667]:
for i in dataset.target:
    temp.append(dataset.target_names[i])

In [669]:
df = pd.DataFrame(data = zip(map(lambda x: x.decode('utf8'),dataset.data), temp, dataset.target),columns = ['text','lang', 'target'])

In [670]:
df.head()

,text,lang,target
0,"Contrariamente, proyectos como Wikipedia, Susn...",es,3
1,Para todos los ejemplos citados y sus variante...,es,3
2,Sie lag im September 2018 auf dem fünften Plat...,de,1
3,Según la tesis doctoral de Ortega Soto (2009) ...,es,3
4,モバイルサイトの構築と平行して2012年からAndroid・iOS・Windows・Fire...,ja,6


In [671]:
docs_train, docs_test, y_train, y_test = train_test_split(
    df['text'], df['target'], test_size=0.2)

In [672]:
docs_train.shape

(790,)

In [673]:
docs_test.shape

(198,)

In [674]:
y_train.shape

(790,)

In [675]:
y_test.shape

(198,)

In [676]:
docs_train

47     A Esplanada é o principal local de interação e...
697    Da sich Wikipedia auf Enzyklopädie-Artikel bes...
660    En juillet 2007, Wikipédia a fait l'objet d'un...
746    Em janeiro de 2007, a Wikipédia entrou na list...
964    Bisher haben international mehr als 2,0 Millio...
59     Un contributeur est classé « actif » tant qu'i...
378    Ende 2000/Anfang 2001 wurden Sanger und Wales ...
870    In certain cases, all editors are allowed to s...
5      В 2009 году художник Роб Мэттьюс (англ. Rob Ma...
25     2009年11月、ウォール・ストリート・ジャーナルは、ウィキペディアへの訪問者が増加する一方...
545    Op 17 juni 2015 werd de jaarlijkse Prinses van...
206    Researchers from Washington University develop...
611    2005年から2006年の年頭にかけてウィキペディアの閲覧者は飛躍的に増加し、インターネット...
62     La croissance de Wikipédia a été favorisée par...
859    Obvious vandalism is generally easy to remove ...
462    Bei umstrittenen Entscheidungen wird in der Wi...
215    Alla fine del 2005 è sorta una controversia do...
41     In February 2007, an art

In [677]:
for i in dataset.target[:15]:
    print(dataset.target_names[i])

es
es
de
es
ja
ru
nl
fr
fr
pl
nl
fr
it
en
en


In [678]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [679]:
def create_tfidf(items):
    co_vect = CountVectorizer(analyzer='word')
    train_counts = co_vect.fit_transform(items)
    tf_transformer = TfidfTransformer(use_idf=True).fit(train_counts)
    train_tfidf = tf_transformer.transform(train_counts)
    return (train_tfidf,co_vect,tf_transformer)

In [680]:
process = create_tfidf(docs_train)

In [681]:
from sklearn.naive_bayes import MultinomialNB

In [682]:
clf = MultinomialNB().fit(process[0],y_train)

In [683]:
X_new_counts = process[1].transform(docs_test)
X_new_tfidf = process[2].transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

In [684]:
# Print the classification report
from sklearn.metrics import classification_report
print (classification_report(y_test, predicted, target_names=dataset.target_names))

# Plot the confusion matrix
cm = metrics.confusion_matrix(y_test, predicted)
print(cm)

              precision    recall  f1-score   support

          ar       1.00      0.17      0.29         6
          de       0.45      1.00      0.62        20
          en       0.87      1.00      0.93        27
          es       1.00      0.97      0.98        32
          fr       1.00      1.00      1.00        25
          it       1.00      1.00      1.00        14
          ja       1.00      0.06      0.11        17
          nl       1.00      1.00      1.00        11
          pl       1.00      0.45      0.62        11
          pt       1.00      1.00      1.00        23
          ru       1.00      1.00      1.00        12

   micro avg       0.86      0.86      0.86       198
   macro avg       0.94      0.79      0.78       198
weighted avg       0.93      0.86      0.83       198

[[ 1  5  0  0  0  0  0  0  0  0  0]
 [ 0 20  0  0  0  0  0  0  0  0  0]
 [ 0  0 27  0  0  0  0  0  0  0  0]
 [ 0  0  1 31  0  0  0  0  0  0  0]
 [ 0  0  0  0 25  0  0  0  0  0  0]
 [ 0  0

In [692]:
# Predict the result on some short new sentences:
sentences = [
    'This is a language detection test.',
    'Ceci est un test de d\xe9tection de la langue.',
    'Dies ist ein Test, um die Sprache zu erkennen.',
    'Sprachen Sie',
    'Zuppa di funghi'
]
se_df = pd.DataFrame(sentences, columns = ['text'])

se_counts = process[1].transform(se_df['text'])
se_tfidf = process[2].transform(se_counts)

predicted = clf.predict(se_tfidf)

for s, p in zip(se_df['text'], predicted):
    print('The language of "%s" is "%s"' % (s, dataset.target_names[p]))

The language of "This is a language detection test." is "en"
The language of "Ceci est un test de détection de la langue." is "fr"
The language of "Dies ist ein Test, um die Sprache zu erkennen." is "de"
The language of "Sprachen Sie" is "de"
The language of "Zuppa di funghi" is "it"


In [686]:
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])
text_clf.fit(docs_train, y_train) 

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [687]:
predicted = text_clf.predict(docs_test)

In [688]:
# Print the classification report
print(metrics.classification_report(y_test, predicted,
                                    target_names=dataset.target_names))

# Plot the confusion matrix
cm = metrics.confusion_matrix(y_test, predicted)
print(cm)

              precision    recall  f1-score   support

          ar       1.00      0.17      0.29         6
          de       0.45      1.00      0.62        20
          en       0.87      1.00      0.93        27
          es       1.00      0.97      0.98        32
          fr       1.00      1.00      1.00        25
          it       1.00      1.00      1.00        14
          ja       1.00      0.06      0.11        17
          nl       1.00      1.00      1.00        11
          pl       1.00      0.45      0.62        11
          pt       1.00      1.00      1.00        23
          ru       1.00      1.00      1.00        12

   micro avg       0.86      0.86      0.86       198
   macro avg       0.94      0.79      0.78       198
weighted avg       0.93      0.86      0.83       198

[[ 1  5  0  0  0  0  0  0  0  0  0]
 [ 0 20  0  0  0  0  0  0  0  0  0]
 [ 0  0 27  0  0  0  0  0  0  0  0]
 [ 0  0  1 31  0  0  0  0  0  0  0]
 [ 0  0  0  0 25  0  0  0  0  0  0]
 [ 0  0

In [690]:
# Predict the result on some short new sentences:
sentences = [
    'This is a language detection test.',
    'Ceci est un test de d\xe9tection de la langue.',
    'Dies ist ein Test, um die Sprache zu erkennen.',
    'Sprachen Sie',
    'Zuppa di funghi'
]
predicted = text_clf.predict(sentences)
for s, p in zip(sentences, predicted):
    print('The language of "%s" is "%s"' % (s, dataset.target_names[p]))

The language of "This is a language detection test." is "en"
The language of "Ceci est un test de détection de la langue." is "fr"
The language of "Dies ist ein Test, um die Sprache zu erkennen." is "de"
The language of "Sprachen Sie" is "de"
The language of "Zuppa di funghi" is "it"
